In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as DT
import yfinance as yf
import numpy as np 
import scipy.stats as stats
from numba import jit
from sklearn.linear_model import LinearRegression
import pandas_market_calendars as mcal
import datetime
from mpl_toolkits.mplot3d import axes3d
import scipy as sp
import scipy.interpolate
from scipy.stats import norm
import math
from nelson_siegel_svensson.calibrate import calibrate_ns_ols 
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

In [7]:
# Define file paths for GDP and CPI data
files = {
    "Eurozone": {
        "GDP": "../Data CSV/GDP Real USD Eurozone Quarterly Economic Time Series Profile.xls",
        "CPI": "../Data CSV/CPI Y o Y Eurozone Monthly Economic Time Series Profile.xls",
    },
    "Switzerland": {
        "GDP": "../Data CSV/GDP Real USD Switzerland Quarterly Economic Time Series Profile.xls",
        "CPI": "../Data CSV/CPI Y o Y Switzerland Monthly Economic Time Series Profile.xls",
    },
    "USA": {
        "GDP": "../Data CSV/GDP Real USD United States of America Quarterly Economic Time Series Profile.xls",
        "CPI": "../Data CSV/CPI Y o Y United States of America Monthly Economic Time Series Profile.xls",
    },
}

store = {}
for country, paths in files.items():
    print(country)

    # Read GDP and CPI data
    gdp_df = pd.read_excel(paths["GDP"], sheet_name="Series Values", header=6)
    cpi_df = pd.read_excel(paths["CPI"], sheet_name="Series Values", header=6)

    # Ensure 'Period' is in datetime format
    gdp_df['Period'] = pd.to_datetime(gdp_df['Period'], errors='coerce')
    cpi_df['Period'] = pd.to_datetime(cpi_df['Period'], errors='coerce')

    # Select relevant columns
    gdp_df = gdp_df[['Period', 'Y-o-Y Change %']].rename(columns={'Y-o-Y Change %': 'GDP_YoY%'})
    cpi_df = cpi_df[["Period", "Value"]].rename(columns={'Value': 'CPI_YoY%'})

    # Define the date range as pandas Timestamps
    start_date = pd.Timestamp("2000-03-31")
    end_date = pd.Timestamp("2024-08-31")

    # Filter dataframes within the date range
    gdp_df = gdp_df[(gdp_df['Period'] >= start_date) & (gdp_df['Period'] <= end_date)]
    cpi_df = cpi_df[(cpi_df['Period'] >= start_date) & (cpi_df['Period'] <= end_date)]

    print("GDP DATA FOR " + country)
    print(gdp_df)
    print("CPI DATA FOR " + country)
    print(cpi_df)

    # Merge GDP and CPI data
    store[country] = pd.merge(gdp_df, cpi_df, on='Period', how='outer')

    print(store[country])


Eurozone
GDP DATA FOR Eurozone
       Period GDP_YoY%
1  2024-06-30    0.006
2  2024-03-31   0.0046
3  2023-12-31   0.0063
4  2023-09-30   0.0047
5  2023-06-30   0.0103
..        ...      ...
94 2001-03-31   0.0512
95 2000-12-31   0.0333
96 2000-09-30   0.0392
97 2000-06-30   0.0467
98 2000-03-31   0.0426

[98 rows x 2 columns]
CPI DATA FOR Eurozone
        Period  CPI_YoY%
2   2024-08-31      2.17
3   2024-07-31      2.58
4   2024-06-30      2.52
5   2024-05-31      2.57
6   2024-04-30      2.37
..         ...       ...
291 2000-07-31      2.09
292 2000-06-30      2.14
293 2000-05-31      1.76
294 2000-04-30      1.71
295 2000-03-31      1.96

[294 rows x 2 columns]
        Period GDP_YoY%  CPI_YoY%
0   2000-03-31   0.0426      1.96
1   2000-04-30      NaN      1.71
2   2000-05-31      NaN      1.76
3   2000-06-30   0.0467      2.14
4   2000-07-31      NaN      2.09
..         ...      ...       ...
289 2024-04-30      NaN      2.37
290 2024-05-31      NaN      2.57
291 2024-06-30    

In [8]:
store["USA"]['GDP_YoY%'] = store["USA"]['GDP_YoY%']\
    .str.replace('%', '', regex=False)\
    .str.replace(r'\((.*?)\)', r'-\1', regex=True)\
    .astype(float) / 100


# %%
store["USA"]['GDP_YoY%'] = store["USA"]['GDP_YoY%'].ffill()
store["Eurozone"]['GDP_YoY%'] = store["Eurozone"]['GDP_YoY%'].ffill()
store["Switzerland"]['GDP_YoY%'] = store["Switzerland"]['GDP_YoY%'].ffill()

print(store["Eurozone"])
print(store["USA"])
print(store["Switzerland"])



        Period  GDP_YoY%  CPI_YoY%
0   2000-03-31    0.0426      1.96
1   2000-04-30    0.0426      1.71
2   2000-05-31    0.0426      1.76
3   2000-06-30    0.0467      2.14
4   2000-07-31    0.0467      2.09
..         ...       ...       ...
289 2024-04-30    0.0046      2.37
290 2024-05-31    0.0046      2.57
291 2024-06-30    0.0060      2.52
292 2024-07-31    0.0060      2.58
293 2024-08-31    0.0060      2.17

[294 rows x 3 columns]
        Period  GDP_YoY%  CPI_YoY%
0   2000-03-31    0.0422      3.76
1   2000-04-30    0.0422      3.07
2   2000-05-31    0.0422      3.19
3   2000-06-30    0.0524      3.73
4   2000-07-31    0.0524      3.66
..         ...       ...       ...
289 2024-04-30    0.0290      3.36
290 2024-05-31    0.0290      3.27
291 2024-06-30    0.0304      2.97
292 2024-07-31    0.0304      2.89
293 2024-08-31    0.0304      2.53

[294 rows x 3 columns]
        Period  GDP_YoY%  CPI_YoY%
0   2000-03-31    0.0432      1.48
1   2000-04-30    0.0432      1.40
2   200

/var/folders/29/j6zpgq394917k8prm4sg190h0000gn/T/ipykernel_43042/3099395196.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  store["Eurozone"]['GDP_YoY%'] = store["Eurozone"]['GDP_YoY%'].ffill()
/var/folders/29/j6zpgq394917k8prm4sg190h0000gn/T/ipykernel_43042/3099395196.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  store["Switzerland"]['GDP_YoY%'] = store["Switzerland"]['GDP_YoY%'].ffill()


In [9]:

# Step 1: Fit MRS Model to Inflation Data FOR THE EUROZONE
# Fit a two-regime Markov Switching model to the inflation rate
# Inflation Model
inflation_model = MarkovRegression(store["Eurozone"]['CPI_YoY%'], k_regimes=2, trend='c', switching_variance=True)
inflation_result = inflation_model.fit()

print("Inflation Model Summary:")
print(inflation_result.summary())

# Step 3: Extract Inflation Regime Probabilities
store["Eurozone"]['Low_Inf_Prob'] = inflation_result.smoothed_marginal_probabilities[0]
store["Eurozone"]['High_Inf_Prob'] = inflation_result.smoothed_marginal_probabilities[1]

# Assign Inflation Regimes based on higher probability
store["Eurozone"]['Inflation_Regime'] = np.where( store["Eurozone"]['Low_Inf_Prob'] > store["Eurozone"]['High_Inf_Prob'], 1, 2)


Inflation Model Summary:
                        Markov Switching Model Results                        
Dep. Variable:               CPI_YoY%   No. Observations:                  294
Model:               MarkovRegression   Log Likelihood                -509.757
Date:                Fri, 08 Nov 2024   AIC                           1031.515
Time:                        23:53:10   BIC                           1053.616
Sample:                             0   HQIC                          1040.366
                                - 294                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4501      0.108      4.185      0.000       0.239       0.661
sigma2         0.2712      

In [10]:
# store["Eurozone"]["Inflation_Regime"].value_counts()

filtered_rows = store["Eurozone"][store["Eurozone"]['Inflation_Regime'] == 1]

print(filtered_rows.head(60))


        Period  GDP_YoY%  CPI_YoY%  Low_Inf_Prob  High_Inf_Prob  \
106 2009-01-31   -0.0185      1.13      0.643967       0.356033   
107 2009-02-28   -0.0185      1.18      0.860413       0.139587   
108 2009-03-31   -0.0499      0.57      0.979431       0.020569   
109 2009-04-30   -0.0499      0.61      0.996798       0.003202   
110 2009-05-31   -0.0499      0.04      0.999472       0.000528   
111 2009-06-30   -0.0454     -0.14      0.999765       0.000235   
112 2009-07-31   -0.0454     -0.64      0.999613       0.000387   
113 2009-08-31   -0.0454     -0.16      0.999814       0.000186   
114 2009-09-30   -0.0365     -0.33      0.999774       0.000226   
115 2009-10-31   -0.0365     -0.13      0.999624       0.000376   
116 2009-11-30   -0.0365      0.48      0.998322       0.001678   
117 2009-12-31   -0.0148      0.93      0.988420       0.011580   
118 2010-01-31   -0.0148      0.95      0.952707       0.047293   
119 2010-02-28   -0.0148      0.85      0.828552       0.17144

In [11]:
# Step 2: Fit MRS Model to GDP Data FOR THE EUROZONE
# Fit a two-regime Markov Switching model to the GDP rate
# GDP Model
inflation_model = MarkovRegression(store["Eurozone"]['GDP_YoY%'], k_regimes=2, trend='c', switching_variance=True)
inflation_result = inflation_model.fit()

print("Inflation Model Summary:")
print(inflation_result.summary())

# Step 3: Extract Inflation Regime Probabilities
store["Eurozone"]['Low_Growth_Prob'] = inflation_result.smoothed_marginal_probabilities[0]
store["Eurozone"]['High_Growth_Prob'] = inflation_result.smoothed_marginal_probabilities[1]

# Assign Inflation Regimes based on higher probability
store["Eurozone"]['Growth_Regime'] = np.where( store["Eurozone"]['Low_Growth_Prob'] > store["Eurozone"]['High_Growth_Prob'], 1, 2)


Inflation Model Summary:
                        Markov Switching Model Results                        
Dep. Variable:               GDP_YoY%   No. Observations:                  294
Model:               MarkovRegression   Log Likelihood                 792.730
Date:                Fri, 08 Nov 2024   AIC                          -1573.461
Time:                        23:53:10   BIC                          -1551.359
Sample:                             0   HQIC                         -1564.610
                                - 294                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0176      0.001     19.915      0.000       0.016       0.019
sigma2      6.262e-05   1.1

In [12]:
filtered_rows = store["Eurozone"][store["Eurozone"]['Growth_Regime'] == 2]

print(filtered_rows.count())

Period              102
GDP_YoY%            102
CPI_YoY%            102
Low_Inf_Prob        102
High_Inf_Prob       102
Inflation_Regime    102
Low_Growth_Prob     102
High_Growth_Prob    102
Growth_Regime       102
dtype: int64
